## This File Contains Some Info Reguarding The MAMS For ABSA Repo And How Their Code Works

## This Section Explains Their Preprocess Workflow

Their Whole Config is saved in a yaml format and loaded before preprocessing

In [1]:
import yaml

config = yaml.safe_load(open('config.yml'))

print(type(config))

for key,value in config.items():
    print(key,value, sep=" : " , end='\n')


<class 'dict'>
mode : category
base_path : ./data/MAMS-ACSA
glove_path : ./data/glove.840B.300d.txt
sentiment_path : ./data/sentiment_dict.json
lowercase : True
max_vocab_size : None
min_vocab_freq : 0
aspect_term_model : {'type': 'recurrent_capsnet', 'recurrent_capsnet': {'embed_size': 300, 'dropout': 0.5, 'num_layers': 2, 'capsule_size': 300, 'bidirectional': True, 'optimizer': 'adam', 'batch_size': 64, 'learning_rate': 0.0003, 'weight_decay': 0, 'num_epoches': 20, 'gpu': 0}, 'bert_capsnet': {'bert_size': 768, 'capsule_size': 300, 'dropout': 0.1, 'optimizer': 'adam', 'batch_size': 32, 'learning_rate': 2e-05, 'weight_decay': 0, 'num_epoches': 5, 'gpu': 0}}
aspect_category_model : {'type': 'recurrent_capsnet', 'recurrent_capsnet': {'embed_size': 300, 'dropout': 0.5, 'num_layers': 2, 'capsule_size': 300, 'bidirectional': True, 'optimizer': 'adam', 'batch_size': 64, 'learning_rate': 0.0003, 'weight_decay': 0, 'num_epoches': 20, 'gpu': 0}, 'bert_capsnet': {'bert_size': 768, 'capsule_size'

Now They Start The Preprocessing. The Entire Code is Abstracted into a subroutine, but i will unpack some of it here.

In [2]:
# Some Imports
import os
import numpy
import pickle
import yaml


In [3]:
# From The Config File Get the mode
mode = config['mode']
print(mode)

category


In [4]:
# Check Mode in Term or Category
assert mode in ('term', 'category')


In [5]:
#Get the Data Locations
base_path = config['base_path']
raw_train_path = os.path.join(base_path, 'raw/train.xml')
raw_val_path = os.path.join(base_path, 'raw/val.xml')
raw_test_path = os.path.join(base_path, 'raw/test.xml')
print(base_path,raw_train_path,raw_val_path,raw_test_path,sep=" , ",end='\n')

./data/MAMS-ACSA , ./data/MAMS-ACSA\raw/train.xml , ./data/MAMS-ACSA\raw/val.xml , ./data/MAMS-ACSA\raw/test.xml


In [6]:
# Check If The Config Has Lowercase Enabled
lowercase = config['lowercase']
print(lowercase)

True


In [7]:
# In Case Of mode == Term

# This Parses The Raw XML data Into Usable Data

from xml.etree.ElementTree import parse
tree = parse(raw_train_path)
sentences = tree.getroot()
data = []
split_char = '__split__'

print(tree)
print(sentences)
print(type(sentences))
print(data)

for sentence in sentences:
    text = sentence.find('text')
    if text is None:
        continue
    text = text.text
    if lowercase:
        text = text.lower()
    aspectTerms = sentence.find('aspectTerms')
    if aspectTerms is None:
        continue
    print(text, ' ' , end='')
    for aspectTerm in aspectTerms:
        term = aspectTerm.get('term')
        if lowercase:
            term = term.lower()
        polarity = aspectTerm.get('polarity')
        start = aspectTerm.get('from')
        end = aspectTerm.get('to')
        piece = text + split_char + term + split_char + polarity + split_char + start + split_char + end
        print('Term :', term , ' From ',start, ' To ' , end , ' Polarity ' , polarity)
        data.append(piece)
    print('\n')

print(len(sentences))



<Element 'sentences' at 0x00000223883E5A30>
<class 'xml.etree.ElementTree.Element'>
[]
1728


Now If We Add That Code To A Function That Parses The Raw data according to term

In [8]:
from xml.etree.ElementTree import parse

def parse_sentence_term(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            if lowercase:
                term = term.lower()
            polarity = aspectTerm.get('polarity')
            start = aspectTerm.get('from')
            end = aspectTerm.get('to')
            piece = text + split_char + term + split_char + polarity + split_char + start + split_char + end
            data.append(piece)
    return data

Now If The Mode Was not selected Term But Category instead

In [9]:
# Parse The Raw XML Data according to Category
from xml.etree.ElementTree import parse

tree = parse(raw_train_path)
sentences = tree.getroot()
data = []
split_char = '__split__'
for sentence in sentences:
    text = sentence.find('text')
    if text is None:
        continue
    text = text.text
    if lowercase:
        text = text.lower()
    aspectCategories = sentence.find('aspectCategories')
    if aspectCategories is None:
        continue
    for aspectCategory in aspectCategories:
        category = aspectCategory.get('category')
        polarity = aspectCategory.get('polarity')
        piece = text + split_char + category + split_char + polarity
        data.append(piece)
print(data)
# Nothing is shown as the data selected is term based

['\n   im a more happier person after discovering the i/p button !\n  __split__button__split__positive', "\n   but , if you 're looking for my opinion of the apex dvd player , i love it !\n  __split__player__split__positive", '\n   it practically plays almost everything you give it .\n  __split__player__split__positive', "\n   i 've had the player for about 2 years now and it still performs nicely with the exception of an occasional wwhhhrrr sound from the motor .\n  __split__player__split__positive", "\n   i 've had the player for about 2 years now and it still performs nicely with the exception of an occasional wwhhhrrr sound from the motor .\n  __split__sound__split__negative", '\n   for the price it is a well spent investment !\n  __split__price__split__positive', '\n   this is by far the nicest one , in so many ways .\n  __split__player__split__positive', "\n   it 's very sleek looking with a very good front panel button layout , and it has a great feature set .\n  __split__look__

Now As a Function

In [10]:
def parse_sentence_category(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectCategories = sentence.find('aspectCategories')
        if aspectCategories is None:
            continue
        for aspectCategory in aspectCategories:
            category = aspectCategory.get('category')
            polarity = aspectCategory.get('polarity')
            piece = text + split_char + category + split_char + polarity
            data.append(piece)
    return data


Now Depending On the mode We can parse The data accordingly 

In [11]:
if mode == 'term':
    train_data = parse_sentence_term(raw_train_path, lowercase=lowercase)
    val_data = parse_sentence_term(raw_val_path, lowercase=lowercase)
    test_data = parse_sentence_term(raw_test_path, lowercase=lowercase)
else:
    train_data = parse_sentence_category(raw_train_path, lowercase=lowercase)
    val_data = parse_sentence_category(raw_val_path, lowercase=lowercase)
    test_data = parse_sentence_category(raw_test_path, lowercase=lowercase)

print(len(train_data))
print(len(val_data))
print(len(test_data))

2101
888
901


In [12]:
remove_list = ['conflict']
# now We have a Type To Remove
print(remove_list)

['conflict']


In [13]:
# To Remove The Aspects That results in a conflict
remove_set = set(remove_list)
filtered_data = []
print(test_data[0].split('__split__'))
for text in train_data:
    if not text.split('__split__')[2] in remove_set:
        new_string = text.replace('\n', '')
        filtered_data.append(new_string)
print(len(filtered_data))

for i,data in enumerate(filtered_data):
    print(f'{i}th Data is {data}')

['we went again and sat at the bar this time, i had 5 pints of guinness and not one buy-back, i ordered a basket of onion rings and there were about 5 in the basket, the rest was filled with crumbs, the chili was not even edible.', 'place', 'neutral']
2101
0th Data is    im a more happier person after discovering the i/p button !  __split__button__split__positive
1th Data is    but , if you 're looking for my opinion of the apex dvd player , i love it !  __split__player__split__positive
2th Data is    it practically plays almost everything you give it .  __split__player__split__positive
3th Data is    i 've had the player for about 2 years now and it still performs nicely with the exception of an occasional wwhhhrrr sound from the motor .  __split__player__split__positive
4th Data is    i 've had the player for about 2 years now and it still performs nicely with the exception of an occasional wwhhhrrr sound from the motor .  __split__sound__split__negative
5th Data is    for the price 

Now If We Make it a Subroutine

In [14]:
def category_filter(data, remove_list):
    remove_set = set(remove_list)
    filtered_data = []
    for text in data:
        if not text.split('__split__')[2] in remove_set:
            new_string = text.replace('\n', '')
            filtered_data.append(new_string)
    return filtered_data

And Filter out the unecessary data from our data

In [15]:
train_data = category_filter(train_data, remove_list)
val_data = category_filter(val_data, remove_list)
test_data = category_filter(test_data, remove_list)
print(len(train_data))
print(len(val_data))
print(len(test_data))


2101
888
901


Now The Repo Uses A Custom Class To Make Words 2 Vectors

In [16]:
import operator

# Constrains On The Data
PAD = '<pad>'
UNK = '<unk>'
ASPECT = '<aspect>'

PAD_INDEX = 0
UNK_INDEX = 1
ASPECT_INDEX = 2

INF = 1e9

class Vocab(object):

    def __init__(self):
        self._count_dict = dict()
        self._predefined_list = [PAD, UNK, ASPECT]

    def add(self, word):
        if word in self._count_dict:
            self._count_dict[word] += 1
        else:
            self._count_dict[word] = 1

    def add_list(self, words):
        for word in words:
            self.add(word)

    def get_vocab(self, max_size=None, min_freq=0):
        sorted_words = sorted(self._count_dict.items(), key=operator.itemgetter(1), reverse=True)
        word2index = {}
        for word in self._predefined_list:
            word2index[word] = len(word2index)
        for word, freq in sorted_words:
            if word in word2index:
                continue
            if (max_size is not None and len(word2index) >= max_size) or freq < min_freq:
                word2index[word] = word2index[UNK]
            else:
                word2index[word] = len(word2index)
        index2word = {}
        index2word[word2index[UNK]] = UNK
        for word, index in word2index.items():
            if index == word2index[UNK]:
                continue
            else:
                index2word[index] = word
        return word2index, index2word

Now They Used This Function

In [17]:
import spacy
import re

url = re.compile('(<url>.*</url>)')
# The Repo Used Code Here That was not usuable in the latest library patch and 
# To Do Manual download of the new en_core_web model i needed to do 
# python -m spacy download en_core_web_sm
# that allowed This code to work
spacy_en = spacy.load('en_core_web_sm')

def check(x):
    return len(x) >= 1 and not x.isspace()

def tokenizer(text):
    tokens = [tok.text for tok in spacy_en.tokenizer(url.sub('@URL@', text))]
    return list(filter(check, tokens))

def build_vocab(data, max_size, min_freq):
    if max_size == 'None':
        max_size = None
    vocab = Vocab()
    for piece in data:
        text = piece.split('__split__')[0]
        text = tokenizer(text)
        vocab.add_list(text)
    return vocab.get_vocab(max_size=max_size, min_freq=min_freq)

In [18]:
word2index, index2word = build_vocab(train_data, max_size=config['max_vocab_size'], min_freq=config['min_vocab_freq'])

for x in word2index:
    print(x,end=' , ')

print('\n\n')
for x in index2word:
    print(x, end=' , ')


<pad> , <unk> , <aspect> , the , . , , , and , it , is , to , i , a , of , - , this , with , for , that , in , ) , you , but , not , ( , on , have , very , are , player , my , great , as , 's , use , camera , good , has , can , easy , n't , phone , quality , ' , battery , ! , was , all , " , sound , be , software , one , like , at , features , dvd , so , no , than , or , from , does , would , about , price , only , when , an , had , there , which , up , ; , excellent , get , well , play , if , other , just , will , out , do , also , screen , more , me , pictures , your , by , small , ve , even , picture , nice , ipod , its , any , little , really , life , : , after , size , some , they , t , music , creative , thing , am , light , zen , work , did , best , / , still , read , what , time , apex , because , too , nomad , better , problems , many , first , used , enough , comes , m , -- , long , most , product , ... , much , unit , 3 , were , mp3 , songs , 2 , easily , files , dvds , been

In [19]:
# Make A Folder For the processed Data
if not os.path.exists(os.path.join(base_path, 'processed')):
    os.makedirs(os.path.join(base_path, 'processed'))

Functions To Save Data

In [20]:
import numpy as np
from pytorch_pretrained_bert import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def save_term_data(data, word2index, path):
    dirname = os.path.dirname(path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    sentence = []
    aspect = []
    label = []
    context = []
    bert_token = []
    bert_segment = []
    td_left = []
    td_right = []
    f = lambda x: word2index[x] if x in word2index else word2index[UNK]
    g = lambda x: list(map(f, tokenizer(x)))
    d = {
        'positive': 0,
        'negative': 1,
        'neutral': 2,
        'conflict': 3
    }
    for piece in data:
        text, term, polarity, start, end = piece.split('__split__')
        start, end = int(start), int(end)
        assert text[start: end] == term
        sentence.append(g(text))
        aspect.append(g(term))
        label.append(d[polarity])
        left_part = g(text[:start])
        right_part = g(text[end:])
        context.append(left_part + [ASPECT_INDEX] + right_part)
        bert_sentence = bert_tokenizer.tokenize(text)
        bert_aspect = bert_tokenizer.tokenize(term)
        bert_token.append(bert_tokenizer.convert_tokens_to_ids(['[CLS]'] + bert_sentence + ['[SEP]'] + bert_aspect + ['[SEP]']))
        bert_segment.append([0] * (len(bert_sentence) + 2) + [1] * (len(bert_aspect) + 1))
        td_left.append(g(text[:end]))
        td_right.append(g(text[start:])[::-1])
        assert len(bert_token[-1]) == len(bert_segment[-1])
    max_length = lambda x: max([len(y) for y in x])
    sentence_max_len = max_length(sentence)
    aspect_max_len = max_length(aspect)
    context_max_len = max_length(context)
    bert_max_len = max_length(bert_token)
    td_left_max_len = max_length(td_left)
    td_right_max_len = max_length(td_right)
    num = len(data)
    for i in range(num):
        sentence[i].extend([0] * (sentence_max_len - len(sentence[i])))
        aspect[i].extend([0] * (aspect_max_len - len(aspect[i])))
        context[i].extend([0] * (context_max_len - len(context[i])))
        bert_token[i].extend([0] * (bert_max_len - len(bert_token[i])))
        bert_segment[i].extend([0] * (bert_max_len - len(bert_segment[i])))
        td_left[i].extend([0] * (td_left_max_len - len(td_left[i])))
        td_right[i].extend([0] * (td_right_max_len - len(td_right[i])))
    sentence = np.asarray(sentence, dtype=np.int32)
    aspect = np.asarray(aspect, dtype=np.int32)
    label = np.asarray(label, dtype=np.int32)
    context = np.asarray(context, dtype=np.int32)
    bert_token = np.asarray(bert_token, dtype=np.int32)
    bert_segment = np.asarray(bert_segment, dtype=np.int32)
    td_left = np.asarray(td_left, dtype=np.int32)
    td_right = np.asarray(td_right, dtype=np.int32)
    np.savez(path, sentence=sentence, aspect=aspect, label=label, context=context, bert_token=bert_token, bert_segment=bert_segment,
             td_left=td_left, td_right=td_right)

def save_category_data(data, word2index, path):
    dirname = os.path.dirname(path)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    sentence = []
    aspect = []
    label = []
    bert_token = []
    bert_segment = []
    f = lambda x: word2index[x] if x in word2index else word2index[UNK]
    g = lambda x: list(map(f, tokenizer(x)))
    d = {
        'positive': 0,
        'negative': 1,
        'neutral': 2,
        'conflict': 3
    }
    cd = {
        'food': 0,
        'service': 1,
        'staff': 2,
        'price': 3,
        'ambience': 4,
        'menu': 5,
        'place': 6,
        'miscellaneous': 7
    }
    counter_z = 8
    for piece in data:
        text, category, polarity = piece.split('__split__')
        sentence.append(g(text))
        if category not in cd.values():
            cd[category] = counter_z
            counter_z +=1
        aspect.append(cd[category])
        label.append(d[polarity])
        bert_sentence = bert_tokenizer.tokenize(text)
        bert_aspect = bert_tokenizer.tokenize(category)
        bert_token.append(bert_tokenizer.convert_tokens_to_ids(['[CLS]'] + bert_sentence + ['[SEP]'] + bert_aspect + ['[SEP]']))
        bert_segment.append([0] * (len(bert_sentence) + 2) + [1] * (len(bert_aspect) + 1))
        assert len(bert_token[-1]) == len(bert_segment[-1])
    max_length = lambda x: max([len(y) for y in x])
    sentence_max_len = max_length(sentence)
    bert_max_len = max_length(bert_token)
    num = len(data)
    for i in range(num):
        sentence[i].extend([0] * (sentence_max_len - len(sentence[i])))
        bert_token[i].extend([0] * (bert_max_len - len(bert_token[i])))
        bert_segment[i].extend([0] * (bert_max_len - len(bert_segment[i])))
    sentence = np.asarray(sentence, dtype=np.int32)
    aspect = np.asarray(aspect, dtype=np.int32)
    label = np.asarray(label, dtype=np.int32)
    bert_token = np.asarray(bert_token, dtype=np.int32)
    bert_segment = np.asarray(bert_segment, dtype=np.int32)
    np.savez(path, sentence=sentence, aspect=aspect, label=label, bert_token=bert_token, bert_segment=bert_segment)

In [21]:
if not os.path.exists(os.path.join(base_path, 'processed')):
    os.makedirs(os.path.join(base_path, 'processed'))
    
if mode == 'term':
    save_term_data(train_data, word2index, os.path.join(base_path, 'processed/train.npz'))
    save_term_data(val_data, word2index, os.path.join(base_path, 'processed/val.npz'))
    save_term_data(test_data, word2index, os.path.join(base_path, 'processed/test.npz'))
else:
    save_category_data(train_data, word2index, os.path.join(base_path, 'processed/train.npz'))
    save_category_data(val_data, word2index, os.path.join(base_path, 'processed/val.npz'))
    save_category_data(test_data, word2index, os.path.join(base_path, 'processed/test.npz'))


In [22]:
import json

def load_glove(path, vocab_size, word2index):
    if not os.path.isfile(path):
        raise IOError('Not a file', path)
    glove = np.random.uniform(-0.01, 0.01, [vocab_size, 300])
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            content = line.split(' ')
            if content[0] in word2index:
                glove[word2index[content[0]]] = np.array(list(map(float, content[1:])))
    glove[PAD_INDEX, :] = 0
    return glove

def load_sentiment_matrix(glove_path, sentiment_path):
    sentiment_matrix = np.zeros((3, 300), dtype=np.float32)
    sd = json.load(open(sentiment_path, 'r', encoding='utf-8'))
    sd['positive'] = set(sd['positive'])
    sd['negative'] = set(sd['negative'])
    sd['neutral'] = set(sd['neutral'])
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            content = line.split(' ')
            word = content[0]
            vec = np.array(list(map(float, content[1:])))
            if word in sd['positive']:
                sentiment_matrix[0] += vec
            elif word in sd['negative']:
                sentiment_matrix[1] += vec
            elif word in sd['neutral']:
                sentiment_matrix[2] += vec
    sentiment_matrix -= sentiment_matrix.mean()
    sentiment_matrix = sentiment_matrix / sentiment_matrix.std() * np.sqrt(2.0 / (300.0 + 3.0))
    return sentiment_matrix

def analyze_term(data):
    num = len(data)
    sentence_lens = []
    aspect_lens = []
    log = {'total': num}
    for piece in data:
        text, term, polarity, _, _ = piece.split('__split__')
        sentence_lens.append(len(tokenizer(text)))
        aspect_lens.append(len(tokenizer(term)))
        if not polarity in log:
            log[polarity] = 0
        log[polarity] += 1
    log['sentence_max_len'] = max(sentence_lens)
    log['sentence_avg_len'] = sum(sentence_lens) / len(sentence_lens)
    log['aspect_max_len'] = max(aspect_lens)
    log['aspect_avg_len'] = sum(aspect_lens) / len(aspect_lens)
    return log

def analyze_category(data):
    num = len(data)
    sentence_lens = []
    log = {'total': num}
    for piece in data:
        text, category, polarity = piece.split('__split__')
        sentence_lens.append(len(tokenizer(text)))
        if not polarity in log:
            log[polarity] = 0
        log[polarity] += 1
    log['sentence_max_len'] = max(sentence_lens)
    log['sentence_avg_len'] = sum(sentence_lens) / len(sentence_lens)
    return log


In [23]:
glove = load_glove(config['glove_path'], len(index2word), word2index)
sentiment_matrix = load_sentiment_matrix(config['glove_path'], config['sentiment_path'])
np.save(os.path.join(base_path, 'processed/glove.npy'), glove)
np.save(os.path.join(base_path, 'processed/sentiment_matrix.npy'), sentiment_matrix)
with open(os.path.join(base_path, 'processed/word2index.pickle'), 'wb') as handle:
    pickle.dump(word2index, handle)
with open(os.path.join(base_path, 'processed/index2word.pickle'), 'wb') as handle:
    pickle.dump(index2word, handle)
analyze = analyze_term if mode == 'term' else analyze_category
log = {
    'vocab_size': len(index2word),
    'oov_size': len(word2index) - len(index2word),
    'train_data': analyze(train_data),
    'val_data': analyze(val_data),
    'test_data': analyze(test_data),
    'num_categories': 3
}
if not os.path.exists(os.path.join(base_path, 'log')):
    os.makedirs(os.path.join(base_path, 'log'))
with open(os.path.join(base_path, 'log/log.yml'), 'w') as handle:
    yaml.safe_dump(log, handle, encoding='utf-8', allow_unicode=True, default_flow_style=False)